In [45]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/TSception/skorchL1Norm/'

/content/drive/MyDrive


In [46]:
import sys, importlib
#importlib.reload(sys.modules['grasp.config'])
from grasp.config import data_dir
# orveride the data_dir in config file
#data_dir='/content/drive/MyDrive/data/' # googleDrive

In [47]:
data_dir

'/content/drive/MyDrive/data/'

In [48]:
! pip install mne==0.19.2
! pip install torch
! pip install skorch

In [49]:
import torch
from skorch.helper import predefined_split
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from skorch.callbacks import Callback
import numpy as np
import matplotlib.pyplot as plt

from grasp.TSception.Models import TSception2
from grasp.utils import rawData2,SEEGDataset,set_random_seeds,cuda_or_cup
from grasp.config import activeChannels, root_dir, tmp_dir
from grasp.myskorch import plotPrediction, MyRegressor


In [50]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.TSception.Models'])
importlib.reload(sys.modules['grasp.utils'])
from grasp.TSception.Models import TSception,TSception2
from grasp.utils import rawData2

In [51]:
# import from another folder
import sys
sys.path.insert(1, '/content/drive/MyDrive/examples')
from IMV_LSTM.networks import IMVTensorLSTM

In [52]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(rawData2));

In [53]:
device=cuda_or_cup()
seed = 123456789  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed)


GPU computing:   True


In [54]:
%%capture
# suppress the output
sampling_rate=1000
traindata, valdata, testdata = rawData2('band','all',move2=True)  # (chns, 15000/15001, 118) (channels, time, trials)
##traindata, valdata, testdata = rawData2('raw','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)
trainx, trainy = traindata[:, :-1, :], traindata[:, -1, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-1, :], valdata[:, -1, :]
testx, testy = testdata[:, :-1, :], testdata[:, -1, :]



In [55]:
# (10, 110, 15001)
samples=trainx.shape[0]
chnNum=trainx.shape[1]
step=50 #ms
T=1000 #ms
totalLen=trainx.shape[2] #ms
batch_size=int((totalLen-T)/step) # 280


In [56]:
import sys, importlib
importlib.reload(sys.modules['grasp.utils'])
from grasp.utils import SEEGDataset


In [57]:
import inspect as i
import sys
#sys.stdout.write(i.getsource(SEEGDataset))

In [58]:
train_ds = SEEGDataset(trainx, trainy,T, step)
val_ds = SEEGDataset(valx, valy,T, step)
test_ds = SEEGDataset(testx, testy,T, step)

In [59]:
del traindata, valdata,testdata

In [60]:
'''xx=[]
yy=[]
for sample in range(samples):
    x = np.zeros((batch_size, 1, chnNum, T)) # 4D:(280,1,19,1000ms):(batch_size, planes, height, weight)
    targetd = np.zeros((batch_size,1)) # (280, 1)
    target = np.zeros((batch_size, 1))  # (280, 1)

    # format 1 trial into 3D tensor
    # result: regress to force derative not good at all
    for bs in range(batch_size):
        x[bs, 0, :, :] = trainx[0, :, bs*step:(bs*step + T)]
        target[bs, 0] = trainy[0, bs * step + T + 1] # force
    xx.append(x)
    yy.append(target)
xx=np.asarray(xx).astype(np.float32) # (118, 28, 1, 110, 1000)
yy=np.asarray(yy).astype(np.float32) # (118, 28, 1)
'''

'xx=[]\nyy=[]\nfor sample in range(samples):\n    x = np.zeros((batch_size, 1, chnNum, T)) # 4D:(280,1,19,1000ms):(batch_size, planes, height, weight)\n    targetd = np.zeros((batch_size,1)) # (280, 1)\n    target = np.zeros((batch_size, 1))  # (280, 1)\n\n    # format 1 trial into 3D tensor\n    # result: regress to force derative not good at all\n    for bs in range(batch_size):\n        x[bs, 0, :, :] = trainx[0, :, bs*step:(bs*step + T)]\n        target[bs, 0] = trainy[0, bs * step + T + 1] # force\n    xx.append(x)\n    yy.append(target)\nxx=np.asarray(xx).astype(np.float32) # (118, 28, 1, 110, 1000)\nyy=np.asarray(yy).astype(np.float32) # (118, 28, 1)\n'

In [61]:
from grasp.config import *
# define in config file
#num_T = 3,num_S = 3,hidden_size=222,dropout=0.5,Lambda = 1e-6,learning_rate=0.002

model=TSception2(T, step, sampling_rate,chnNum, num_T, num_S,batch_size,dropout).float()

#if(enable_cuda):
#	model.cuda()
 
#checkpoint = torch.load(result_dir+'checkpoint195.pth')
#net.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [62]:

'''
# import this class from grasp.myskorch
class plotPrediction(Callback):
    def on_epoch_begin(self, net,dataset_train=None, dataset_valid=None, **kwargs):
        preds.clear()
        targets.clear()

    def on_batch_end(self, net, X=None, y=None, training=None, **kwargs):
        if training==False:
            target=y.squeeze().numpy()
            step=kwargs
            loss=step['loss']
            y_pred=step['y_pred']
            y_pred=y_pred.squeeze().cpu().detach().numpy()
            preds.append(y_pred)
            targets.append(target)

    def on_epoch_end(self, net,dataset_train=None, dataset_valid=None, **kwargs):
        fig, ax = plt.subplots(figsize=(6, 3))
        plt.ion()
        ax.clear()
        a=np.concatenate(targets)
        b=np.concatenate(preds)
        ax.plot(a, label="True", linewidth=1)
        ax.plot(b, label='Predicted - Test', linewidth=1)
        ax.legend(loc='upper left')
        figname = result_dir + 'prediction' + str(len(net.history)) + '.png'
        fig.savefig(figname)
        plt.close(fig)
'''

'\n# import this class from grasp.myskorch\nclass plotPrediction(Callback):\n    def on_epoch_begin(self, net,dataset_train=None, dataset_valid=None, **kwargs):\n        preds.clear()\n        targets.clear()\n\n    def on_batch_end(self, net, X=None, y=None, training=None, **kwargs):\n        if training==False:\n            target=y.squeeze().numpy()\n            step=kwargs\n            loss=step[\'loss\']\n            y_pred=step[\'y_pred\']\n            y_pred=y_pred.squeeze().cpu().detach().numpy()\n            preds.append(y_pred)\n            targets.append(target)\n\n    def on_epoch_end(self, net,dataset_train=None, dataset_valid=None, **kwargs):\n        fig, ax = plt.subplots(figsize=(6, 3))\n        plt.ion()\n        ax.clear()\n        a=np.concatenate(targets)\n        b=np.concatenate(preds)\n        ax.plot(a, label="True", linewidth=1)\n        ax.plot(b, label=\'Predicted - Test\', linewidth=1)\n        ax.legend(loc=\'upper left\')\n        figname = result_dir + \

In [63]:
net = MyRegressor(
    model,
    #train_split=predefined_split(valid_set),
    iterator_train__shuffle=True,
    train_split=predefined_split(val_ds),
    max_epochs=200,
    lr=learning_rate,
    batch_size=1,
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss,
    callbacks=[('plotPrediction', plotPrediction(result_dir)),],
    device = device
)


In [64]:
net.fit(train_ds,y=None)

  epoch    train_loss    valid_loss       dur
-------  ------------  ------------  --------
      1        0.1258        0.1068  114.9223
      2        0.1160        0.1079  110.3851
      3        0.1144        0.1078  110.3644
      4        0.1139        0.1084  110.3514
      5        0.1138        0.1081  110.3425
      6        0.1139        0.1036  110.4386
      7        0.1136        0.1067  110.4583


<class 'grasp.myskorch.MyRegressor'>[initialized](
  module_=TSception2(
    (Tception1): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 500), stride=(1, 1), padding=(0, 250))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception2): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 250), stride=(1, 1), padding=(0, 125))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception3): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 126), stride=(1, 1), padding=(0, 63))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception4): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 62), stride=(1, 1), padding=(0, 31))
      (1): ReLU()
      (2): AvgPool2d(kernel_size=(1, 16), stride=(1, 8), padding=0)
    )
    (Tception5): Sequential(
      (0): Conv2d(1, 3, kernel_size=(1, 32), stride=(1, 1), padding=(0, 16))
      (1): ReLU()
      